In [1]:
from dotenv import load_dotenv
import os
import pygsheets
import requests
import pandas as pd
from tqdm import tqdm

from pangres import upsert
from sqlalchemy import text, create_engine

load_dotenv()

True

In [ ]:
db_username=os.environ.get("db_username")
db_password=os.environ.get("db_password")
db_host=os.environ.get("db_host")
db_port=os.environ.get("db_port")
db_name=os.environ.get("db_name")


def create_db_connection_string(db_username: str, db_password: str, db_host: str, db_port: int, db_name: str):
    connection_url = f"postgresql+psycopg2://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}"
    return connection_url

## Functions

In [4]:
def get_puuid(gameName, tagline, api_key):
    link = f'https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{username}/{tagline}?api_key={api_key}'

    response = requests.get(link)
    if response.status_code == 200:
        return response.json()['puuid']
    else:
        print(f"Error: {response.status_code}")
    return None

In [5]:
def account_info_by_puuid(puuid, api_key):
    link = f'https://americas.api.riotgames.com/riot/account/v1/accounts/by-puuid/{puuid}?api_key={api_key}'

    response = requests.get(link)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
    return None

In [6]:
def retrieve_match_ids(puuid, api_key, type='ranked', start=0, count=100):
    link = f'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type={type}&start={start}&count={count}&api_key={api_key}'

    response = requests.get(link)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
    return None

In [7]:
def retrieve_match_data(match_id, api_key):
    link = f'https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}'
    response = requests.get(link)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
    return response.json()

In [8]:
def process_match_json(match_id, puuid, api_key):

    side_dict = {
        100:'blue',
        200:'red'
    }

    game = retrieve_match_data(match_id, api_key)

    metadata = game['metadata']
    data_version = metadata['dataVersion']
    match_id = metadata['matchId']
    participants_puuid = metadata['participants']

    match_info = game['info']
    match_duration = match_info['gameDuration'] # 
    match_mode = match_info['gameMode']
    match_type = match_info['gameType']
    match_version = match_info['gameVersion']
    match_map_id = match_info['mapId']
    match_queue_id = match_info['queueId']
    participant_data = match_info['participants']


    main_player = participant_data[participants_puuid.index(puuid)]
    champion_level = main_player['champLevel']
    champion_experience = main_player['champExperience']
    champion_name = main_player['championName']
    champion_id = main_player['championId']
    deaths = main_player['deaths']
    kills = main_player['kills']
    assists = main_player['assists']
    gold_earned = main_player['goldEarned']
    gold_spent = main_player['goldSpent']
    individual_position = main_player['individualPosition']
    item_0 = main_player['item0']
    item_1 = main_player['item1']
    item_2 = main_player['item2']
    item_3 = main_player['item3']
    item_4 = main_player['item4']
    item_5 = main_player['item5']
    item_6 = main_player['item6']
    lane = main_player['lane']
    neutral_minions_killed = main_player['neutralMinionsKilled']
    objectives_stolen = main_player['objectivesStolen']
    participant_id = main_player['participantId']
    riot_id_game_name = main_player['riotIdGameName']
    riot_id_tagline = main_player['riotIdTagline']
    role = main_player['role']
    total_damage_dealt = main_player['totalDamageDealt']
    total_damage_dealt_to_champions = main_player['totalDamageDealtToChampions']
    total_damage_shielded_on_teammates = main_player['totalDamageShieldedOnTeammates']
    total_damage_taken = main_player['totalDamageTaken']
    totalHealsOnTeammates = main_player['totalHealsOnTeammates']
    total_minions_killed = main_player['totalMinionsKilled']
    total_time_cc_dealt = main_player['totalTimeCCDealt']
    team_id = side_dict[main_player['teamId']]
    team_position = main_player['teamPosition']
    turret_kills = main_player['turretKills']
    vision_score = main_player['visionScore']
    vision_wards_bought = main_player['visionWardsBoughtInGame']
    wards_placed = main_player['wardsPlaced']
    wards_killed = main_player['wardsKilled']
    win = main_player['win']

    # perks
    main_player_perks = main_player['perks']
    stats_perks = main_player_perks['statPerks']
    perk_defense = stats_perks['defense']
    perk_flex = stats_perks['flex']
    perk_offense = stats_perks['offense']

    perk_style_selection = main_player_perks['styles']

    primary_style = perk_style_selection[0]
    primary_style_selections = primary_style['selections']
    keystone = primary_style_selections[0]['perk']
    primary_style_selection_1 = primary_style_selections[1]['perk']
    primary_style_selection_2 = primary_style_selections[2]['perk']
    primary_style_selection_3 = primary_style_selections[3]['perk']

    secondary_style = perk_style_selection[1]
    secondary_style_selections = secondary_style['selections']
    secondary_style_selection_1 = secondary_style_selections[0]['perk']
    secondary_style_selection_2 = secondary_style_selections[1]['perk']

    baron_kills = main_player['baronKills']
    dragon_kills = main_player['dragonKills']
    firstBlood = main_player['firstBloodKill']
    firstBloodAssist = main_player['firstBloodAssist']
    firstTower = main_player['firstTowerKill']
    firstTowerAssist = main_player['firstTowerAssist']


    match_dataframe = pd.DataFrame(
        {
            'data_version': [data_version],
            'match_id': [match_id],
            'puuid': [puuid],
            'participants_puuid': [participants_puuid],
            'match_mode': [match_mode],
            'match_type': [match_type],
            'match_duration': [match_duration],
            'match_version': [match_version],
            'match_map_id': [match_map_id],
            'match_queue_id': [match_queue_id],
            'champion_level': [champion_level],
            'champion_experience': [champion_experience],
            'champion_name': [champion_name],
            'champion_id': [champion_id],
            'deaths': [deaths],
            'kills': [kills],
            'assists': [assists],
            'gold_earned': [gold_earned],
            'gold_spent': [gold_spent],
            'individual_position': [individual_position],
            'item_0': [item_0],
            'item_1': [item_1],
            'item_2': [item_2],
            'item_3': [item_3],
            'item_4': [item_4],
            'item_5': [item_5],
            'item_6': [item_6],
            'lane': [lane],
            'neutral_minions_killed': [neutral_minions_killed],
            'objectives_stolen': [objectives_stolen],
            'participant_id': [participant_id],
            'riot_id_game_name': [riot_id_game_name],
            'riot_id_tagline': [riot_id_tagline],
            'role': [role],
            'total_damage_dealt': [total_damage_dealt],
            'total_damage_dealt_to_champions': [total_damage_dealt_to_champions],
            'total_damage_shielded_on_teammates': [total_damage_shielded_on_teammates],
            'total_damage_taken': [total_damage_taken],
            'totalHealsOnTeammates': [totalHealsOnTeammates],
            'total_minions_killed': [total_minions_killed],
            'total_time_cc_dealt': [total_time_cc_dealt],
            'team_id': [team_id],
            'team_position': [team_position],
            'turret_kills': [turret_kills],
            'vision_score': [vision_score],
            'vision_wards_bought': [vision_wards_bought],
            'wards_placed': [wards_placed],
            'wards_killed': [wards_killed],
            'win': [win],
            'keystone': [keystone],
            'primary_style_selection_1': [primary_style_selection_1],
            'primary_style_selection_2': [primary_style_selection_2],
            'primary_style_selection_3': [primary_style_selection_3],
            'secondary_style_selection_1': [secondary_style_selection_1],
            'secondary_style_selection_2': [secondary_style_selection_2],
            'perk_defense': [perk_defense],
            'perk_flex': [perk_flex],
            'perk_offense': [perk_offense],
            'baron_kills': [baron_kills],
            'dragon_kills': [dragon_kills],
            'firstBlood': [firstBlood],
            'firstBloodAssist': [firstBloodAssist],
            'firstTower': [firstTower],
            'firstTowerAssist': [firstTowerAssist],
        }
    )
    return match_dataframe

In [ ]:
# TODO move this to a separate file
UHE_varsity_2025_2026 = {
    'hawnlilmonster': '0528', #TOP
    'bruhdun': '249', #JUNGLE
    'oanduh': 'oan', #MIDDLE
    'RoboBoto': 'NA1', #BOTTOM
    'mazsu': 'mas' #UTILITY
}

In [50]:
username = 'hawnlilmonster'
tagline = '0528'

api_key = os.getenv("riot_api_key")
puuid = get_puuid(username, tagline, api_key)

account_info_by_puuid(puuid, api_key)
match_ids = retrieve_match_ids(puuid, api_key, count=20)

dataframe_list = []

for match_id in tqdm(match_ids):
    match_dataframe = process_match_json(match_id, puuid, api_key)
    dataframe_list.append(match_dataframe)



100%|██████████| 20/20 [00:05<00:00,  3.52it/s]


In [51]:
all_matches_dataframe = pd.concat(dataframe_list)

In [52]:
def retrieve_perk_info():
    perks = requests.get('https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/perks.json').json()
    perkstyles = requests.get('https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/perkstyles.json').json()
    
    perk_dict = {}
    for i in perkstyles['styles']:
        perk_dict[i['id']] = i['name']

    for i in perks:
        perk_dict[i['id']] = i['name']
    return perk_dict

In [53]:
perk_dict = retrieve_perk_info()
perk_columns = ['keystone', 'primary_style_selection_1', 'primary_style_selection_2', 
                'primary_style_selection_3', 'secondary_style_selection_1', 
                'secondary_style_selection_2', 'perk_defense', 'perk_flex', 'perk_offense']

all_matches_dataframe[perk_columns] = all_matches_dataframe[perk_columns].replace(perk_dict)

In [54]:
def retrieve_items_info():
    items = requests.get('https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/items.json').json()
    
    items_dict = {}
    for i in items:
        items_dict[i['id']] = i['name']
    return items_dict

In [55]:
items_dict = retrieve_items_info()
item_columns = ['item_0', 'item_1', 'item_2', 'item_3', 'item_4', 'item_5', 'item_6']
all_matches_dataframe[item_columns] = all_matches_dataframe[item_columns].replace(items_dict)

In [56]:
all_matches_dataframe['username'] = [username]*len(all_matches_dataframe)
all_matches_dataframe['tagline'] = [tagline]*len(all_matches_dataframe)

In [57]:
all_matches_dataframe['uuid'] = all_matches_dataframe['match_id'] + '_' + all_matches_dataframe['puuid']
all_matches_dataframe = all_matches_dataframe.set_index('uuid')

In [58]:
connection_url = create_db_connection_string(db_username, db_password, db_host, db_port, db_name)
db_engine = create_engine(connection_url, pool_recycle=3600)
connection = db_engine.connect()
upsert(con=connection, df=all_matches_dataframe, schema='soloq', table_name='player_matches', create_schema=True, if_row_exists='update')

In [59]:
connection.commit()